PRACTICA REALIZADA POR:

 * Manuel Cubertorer Gumbau
 * Francisco Javier Corrales Estrella
 
En la asignatura de Tipologia y ciclo de vida de los datos de Universidad Oberta Catalunya
Octubre 2021

**Evaluación inicial**

* ROBOTS.txt:
En primer lugar inspeccionamos el fichero robots.txt para ver la política sobre rastreos. El fichero no existe por lo que entendemos que el propietario del sitio web permite rastrear toda la página sin ningún tipo de restricción.

* Mapa del sitio web (sitemap.xml).

* Tamaño

* Tecnología usada


In [34]:

import builtwith
builtwith.parse('https://demanda.ree.es/visiona/peninsula/demanda/tablas/2021-10-23/1')


{'javascript-frameworks': ['AngularJS', 'Modernizr', 'jQuery'],
 'javascript-graphics': ['D3', 'Highcharts'],
 'font-scripts': ['Google Font API'],
 'web-frameworks': ['Twitter Bootstrap']}

In [13]:
import requests

# Petición a la página web
#url principal
#TODO Navegar desde la pagina principal al dia de hoy
#url = "https://demanda.ree.es/visiona/peninsula/demanda/total"
url = "https://demanda.ree.es/visiona/peninsula/demanda/tablas/2021-10-23/1"
page = requests.get(url)

# Lectura del html del response
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content)


In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd

#opciones de navegacion
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

driver_path = '/home/fjcorrales/Master Ciencia de datos/tipologia y ciclo de vida de los datos/PRACTICA-1/lib/chromedriver_linux64/chromedriver'

#Abrimos el navegador
driver = webdriver.Chrome(driver_path)

#Navegamos a la url
driver.get(url)

#Navegamos por los 30 dias anteriores
for i in range(30):            
      
    # Leemos la información por horas    
    for t in range(2,146):
        #dia hora
        xpath_diahora = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[1]"
        element_diahora = driver.find_element_by_xpath(xpath_diahora)
        dia = element_diahora.text.split()[0]
        hora = element_diahora.text.split()[1]

        #Real
        xpath_real = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[2]"
        element_real = driver.find_element_by_xpath(xpath_real)
        lectura_real = element_real.text

        #Prevista
        xpath_prevista = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[3]"
        element_prevista = driver.find_element_by_xpath(xpath_prevista)
        lectura_prevista = element_prevista.text

        #Programada
        xpath_programada = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[4]"
        element_programada = driver.find_element_by_xpath(xpath_programada)
        lectura_programada = element_programada.text

        print(dia, hora, lectura_real, lectura_prevista,lectura_programada)  

    # Pulsamos click para navegar al dia anterior
    wait = WebDriverWait(driver, 5)
    elementButton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.md-icon-button main-menu md-button md-ink-ripple'.replace(' ','.'))))
    elementButton.click()




<ipython-input-175-490a287d1d3a>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)
<ipython-input-175-490a287d1d3a>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_diahora = driver.find_element_by_xpath(xpath_diahora)
<ipython-input-175-490a287d1d3a>:34: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_real = driver.find_element_by_xpath(xpath_real)
<ipython-input-175-490a287d1d3a>:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_prevista = driver.find_element_by_xpath(xpath_prevista)
<ipython-input-175-490a287d1d3a>:44: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_programada = driver.find_element_by_xpath(xpath_programada)


2021-10-22 21:00 29.690 29.744 29.024
2021-10-22 21:10 29.426 29.470 29.024
2021-10-22 21:20 29.186 29.171 29.024
2021-10-22 21:30 28.958 28.848 29.024
2021-10-22 21:40 28.416 28.501 29.024
2021-10-22 21:50 28.107 28.130 29.024
2021-10-22 22:00 27.253 27.733 26.637
2021-10-22 22:10 27.143 27.327 26.637
2021-10-22 22:20 26.598 26.922 26.637
2021-10-22 22:30 26.403 26.519 26.637
2021-10-22 22:40 25.989 26.118 26.637
2021-10-22 22:50 25.732 25.719 26.637
2021-10-22 23:00 25.464 25.322 24.396
2021-10-22 23:10 24.930 24.959 24.396
2021-10-22 23:20 24.969 24.661 24.396
2021-10-22 23:30 24.449 24.428 24.396
2021-10-22 23:40 24.348 24.260 24.396
2021-10-22 23:50 24.220 24.158 24.396
2021-10-23 00:00 23.918 24.120 23.699
2021-10-23 00:10 24.216 24.094 23.699
2021-10-23 00:20 24.291 24.024 23.699
2021-10-23 00:30 23.936 23.910 23.699
2021-10-23 00:40 23.928 23.753 23.699
2021-10-23 00:50 23.446 23.552 23.699
2021-10-23 01:00 23.065 23.307 22.514
2021-10-23 01:10 23.181 23.049 22.514
2021-10-23 0

2021-10-22 09:00 29.048 29.404 29.756
2021-10-22 09:10 29.406 29.420 29.756
2021-10-22 09:20 29.567 29.448 29.756
2021-10-22 09:30 29.619 29.486 29.756
2021-10-22 09:40 29.557 29.535 29.756
2021-10-22 09:50 29.901 29.596 29.756
2021-10-22 10:00 29.662 29.667 29.666
2021-10-22 10:10 29.768 29.740 29.666
2021-10-22 10:20 29.789 29.802 29.666
2021-10-22 10:30 29.809 29.855 29.666
2021-10-22 10:40 29.764 29.898 29.666
2021-10-22 10:50 29.947 29.932 29.666
2021-10-22 11:00 29.848 29.955 30.145
2021-10-22 11:10 29.917 29.974 30.145
2021-10-22 11:20 29.854 29.994 30.145
2021-10-22 11:30 29.769 30.013 30.145
2021-10-22 11:40 29.879 30.033 30.145
2021-10-22 11:50 29.953 30.053 30.145
2021-10-22 12:00 29.841 30.073 30.216
2021-10-22 12:10 30.086 30.092 30.216
2021-10-22 12:20 30.118 30.110 30.216
2021-10-22 12:30 30.039 30.127 30.216
2021-10-22 12:40 30.181 30.142 30.216
2021-10-22 12:50 30.277 30.155 30.216
2021-10-22 13:00 30.236 30.167 30.191
2021-10-22 13:10 30.054 30.164 30.191
2021-10-22 1

2021-10-19 21:00 30.830 31.104 30.100
2021-10-19 21:10 30.871 30.786 30.100
2021-10-19 21:20 30.668 30.442 30.100
2021-10-19 21:30 30.177 30.071 30.100
2021-10-19 21:40 29.911 29.673 30.100
2021-10-19 21:50 29.198 29.249 30.100
2021-10-19 22:00 28.778 28.798 27.618
2021-10-19 22:10 28.526 28.345 27.618
2021-10-19 22:20 28.117 27.915 27.618
2021-10-19 22:30 27.812 27.505 27.618
2021-10-19 22:40 27.004 27.118 27.618
2021-10-19 22:50 27.030 26.752 27.618
2021-10-19 23:00 26.379 26.407 25.323
2021-10-19 23:10 25.899 26.093 25.323
2021-10-19 23:20 25.908 25.817 25.323
2021-10-19 23:30 25.484 25.580 25.323
2021-10-19 23:40 24.922 25.380 25.323
2021-10-19 23:50 24.934 25.218 25.323
2021-10-20 00:00 24.615 25.095 24.637
2021-10-20 00:10 25.036 24.979 24.637
2021-10-20 00:20 24.864 24.840 24.637
2021-10-20 00:30 24.641 24.678 24.637
2021-10-20 00:40 24.565 24.493 24.637
2021-10-20 00:50 24.309 24.286 24.637
2021-10-20 01:00 23.825 24.054 23.319
2021-10-20 01:10 23.721 23.819 23.319
2021-10-20 0